In [1]:
({get_profile_subset()})

NameError: name 'get_profile_subset' is not defined

In [2]:
import random
import os
from litellm import completion
from datasets import load_dataset

USER_PROFILE = {
    "location": "Cambridge, UK",
    "background": "Master's in Machine Learning, Australian citizen, family in China, girlfriend from Thailand, soon to be quant researcher",
    "skills": ["Python", "Chinese", "Machine Learning", "Statistics", "Web Crawling", "Quant research", "Data Analysis", "Digital electronics", "Arduino and Raspberry Pis", "Tutoring", "Manufacturing", "Soldering", "Engineering", "Fishing", "Machining", "Mushroom Foraging", "Coral Aquaculture", "Baking", "Importing/Exporting","Gardening"],
    "assets": "About 1-2k pounds initially, access to network of engineers, startup founders, friends in China, Thailand, Singapore and Australia",
}

def get_profile_with_skill_subset(n_skills=5):
    """Returns the full profile with a randomized subset of skills."""
    # Ensure we don't try to sample more skills than exist
    k = min(n_skills, len(USER_PROFILE["skills"]))
    
    # Create the output dictionary
    profile_subset = USER_PROFILE.copy()
    profile_subset["skills"] = random.sample(USER_PROFILE["skills"], k)
    
    return profile_subset

print("⚡ Initializing Nemotron-USA Persona Stream...")
ds = load_dataset("nvidia/Nemotron-Personas-USA", split="train", streaming=True)
ds_iter = iter(ds.shuffle(buffer_size=10000))

def get_real_nemotron_persona():
    item = next(ds_iter)
    identity = (
        f"Occupation: {item['occupation']}. "
        f"Persona: {item['persona']}. "
        f"Traits: Age {item['age']}, Sex {item['sex']}, Education {item['education_level']}, Marital Status {item['marital_status']}."
        f"Skills: {item['skills_and_expertise']}. "
        f"Hobbies: {item['hobbies_and_interests']}."
    )
    return {"name": f"Citizen_{random.randint(100, 999)}", "prompt": identity}

LENSES = [
    {"name": "Economic Viability", "prompt": "Analyze unit economics, market size, and ROI."},
    {"name": "Technical Feasibility", "prompt": "Analyze if the founder's skills can actually build this. Be brutally honest and critical."},
    {"name": "Ease of Realisation", "prompt": "Analyze how likely the idea will actually succeed with a small starting cost. Be brutally honest and critical."},
    {"name": "Potential Problems", "prompt": "Analyze all the potential problems with this idea. Be brutally honest and critical."},
    {"name": "Market Competition", "prompt": "Analyze whether the market is already saturated with alternatives."},
    {"name": "Identify Most Grounded", "prompt": "Identify the most grounded, realistic and viable ideas from the list of ideas."}
]

def run_triple_rotation_factory(max_rounds=12):
    discussion = []
    
    for r in range(1, max_rounds + 1):
        is_persona_round = not (r % 3 == 2) # Every 3rd round is a lens round, the rest are persona seeding rounds
        mode_label = "🎭 PERSONA SEEDING" if is_persona_round else "🔍 LENS FILTERING"
        
        print(f"\n--- ROUND {r} [{mode_label}] ---")
        
        current_squad = [get_real_nemotron_persona() for _ in range(4)] if is_persona_round else LENSES

        for agent in current_squad:
            context = "\n".join(discussion[-16:]) # Rolling context window
            
            if is_persona_round:
                if r <= 2:
                    instruction = (
                        f"Given the founder's profile ({get_profile_with_skill_subset()}), generate as many new business ideas as possible."
                        "Ideas have to be highly likely to actually work in the real world, and as simple as possible to get started with under £2k."
                        "Try to focus on buying and selling something."
                    )
                else:
                    instruction = (
                        f"Given the previous ideas generated and the founder's profile ({get_profile_with_skill_subset()}), "
                        "Find weaknesses of previous ideas and generate/refine as many new business ideas as possible."
                        "Ideas have to be highly likely to actually work in the real world, and as simple as possible to get started with under £2k."
                        "Try to focus on buying and selling something."
                    )

            else:
                instruction = (
                    f"Critique all previous ideas using the {agent['name']} lens. "
                    f"Filter out anything that is not feasible for the founder: {USER_PROFILE}. "
                    "Only allow the most viable, high-potential ideas to remain."
                )

            sys_prompt = f"{agent['prompt']}\n{instruction}"
            
            response = completion(
                model="openai/gpt-4o",
                messages=[
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": f"History:\n{context}\n\nYour contribution:"}
                ],
                temperature=1.2 if is_persona_round else 0.3 
            )
            
            ans = response.choices[0].message.content
            label = agent.get('name', "Agent")
            print(f"✅ [{label}]: {ans}...")
            discussion.append(f"[{label}]: {ans}")

    summary = completion(
        model="openai/gpt-4o",
        messages=[
            {"role": "system", "content": f"You are an analyst. Based on the 12-round debate, identify the single most viable business for this founder with the highest probability of actually succeeding in the real world: {USER_PROFILE}."},
            {"role": "user", "content": "\n".join(discussion)}
        ],
        temperature=0
    ).choices[0].message.content

    return summary

print("\n🏆 FINAL BUSINESS CASE:\n", run_triple_rotation_factory(max_rounds=12))

⚡ Initializing Nemotron-USA Persona Stream...

--- ROUND 1 [🎭 PERSONA SEEDING] ---
✅ [Citizen_475]: 1. **Sustainable Electronics Kits**: Design and sell DIY electronics kits that focus on sustainability. Utilize the founder’s engineering and soldering skills to create educational kits. These kits can focus on teaching essential electronics while integrating components that illustrate renewable energy concepts—such as solar panels or energy efficiency circuits. Leverage both the Australian and Asian markets’ interest in environmental education, utilizing the founder’s diverse cultural network for distribution.

2. **Cultural Food Kits**: Combine the founder’s geographic connections with Helene’s cooking experimentation interest by developing food kits featuring authentic ingredients from Chinese, Thai, and Australian cuisines. These kits could target expatriates missing home or adventurers seeking to explore new cultures through cooking. Start by sourcing key ingredients directly from c

RateLimitError: litellm.RateLimitError: RateLimitError: OpenAIException - Request too large for gpt-4o in organization org-w9DueHlITys3I7s7bQUAgPrX on tokens per min (TPM): Limit 30000, Requested 35479. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.